### Setting Up Your Environment

In [1]:
# Install required packages
#!pip install tensorflow opencv-python matplotlib pandas scikit-learn transformers
#!pip install face-recognition dlib
#!pip install gradio # For creating a simple demo interface

### Data Collection & Preparation

In [2]:
# Download and extract the FER2013 dataset
#!wget https://www.kaggle.com/datasets/msambare/fer2013/download -O fer2013.zip
#!unzip fer2013.zip -d data

### Model Building

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

# Define the CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotions: angry, disgust, fear, happy, sad, surprise, neutral
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

c:\Users\dwipm\.conda\envs\ml-venv-39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def create_and_save_model():
    # Load base model
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

    # Add custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(7, activation='softmax')(x)

    # Create model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Save model
    model.save('emotion_model.h5')
    print("Model created and saved!")

    return model

# Create and save model
model = create_and_save_model()

C:\Users\dwipm\AppData\Local\Temp\ipykernel_13904\1573050073.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


Model created and saved!


### Fine-tuning with Transfer Learning

In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Add custom layers for emotion detection
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

# Create and compile the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\dwipm\AppData\Local\Temp\ipykernel_13904\3093589820.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


### Real-time Facial Emotion Detection

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the pre-trained model
model = load_model('emotion_model.h5')

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_emotion(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Extract face ROI
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = roi_gray.astype('float') / 255.0
        roi_gray = img_to_array(roi_gray)
        roi_gray = np.expand_dims(roi_gray, axis=0)

        # Make prediction
        prediction = model.predict(roi_gray)[0]
        emotion_idx = np.argmax(prediction)
        emotion = emotion_labels[emotion_idx]
        confidence = prediction[emotion_idx] * 100

        # Draw bounding box and emotion label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f"{emotion}: {confidence:.2f}%"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    return frame, emotion if len(faces) > 0 else 'No face detected'

# Function to process webcam feed (for testing locally)
def process_webcam():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame, detected_emotion = detect_emotion(frame)

        cv2.imshow('Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Chatbot Detection

In [8]:
import gradio as gr
import cv2
import numpy as np
import random

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Simple function to detect faces and assign random emotions
def detect_emotion(image):
    if image is None:
        return image, "No image provided"
    
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Load face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    result_image = image.copy()
    
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            # For demo: random emotion
            emotion = random.choice(emotion_labels)
            
            # Draw rectangle around face
            cv2.rectangle(result_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Draw emotion label
            cv2.putText(result_image, emotion, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    else:
        emotion = "No face detected"
    
    return result_image, emotion

# Simple function to generate chatbot responses
def get_response(emotion, message):
    emotion_responses = {
        'Happy': "You seem happy! That's great!",
        'Sad': "I notice you might be feeling down. How can I help?",
        'Angry': "I sense you might be frustrated. Would you like to talk about it?",
        'Surprise': "You look surprised! What's going on?",
        'Neutral': "How can I assist you today?",
        'Fear': "You seem concerned. Is everything okay?",
        'Disgust': "Something seems to be bothering you. Can I help?",
        'No face detected': "I can't see your face clearly. Can you try another image?"
    }
    
    response = emotion_responses.get(emotion, "How can I help you today?")
    
    if message:
        if "hello" in message.lower() or "hi" in message.lower():
            response += " Hello there! What can I do for you?"
        elif "how are you" in message.lower():
            response += " I'm doing well, thanks for asking!"
        elif "bye" in message.lower():
            response += " Goodbye! Have a great day!"
        else:
            response += " Tell me more about how I can help you."
    
    return response

# Chatbot interface function
def chatbot_interface(image, message):
    # Process image to detect emotion
    processed_image, emotion = detect_emotion(image)
    
    # Generate response based on emotion and message
    response = get_response(emotion, message)
    
    return processed_image, response, emotion

# Create a simple Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Simple Emotion Chatbot")
    
    with gr.Row():
        with gr.Column():
            camera_input = gr.Image(source="webcam", label="Camera Input")
            text_input = gr.Textbox(label="Your Message")
            submit_btn = gr.Button("Send")
        
        with gr.Column():
            image_output = gr.Image(label="Emotion Detection")
            text_output = gr.Textbox(label="Chatbot Response")
    
    submit_btn.click(
        fn=chatbot_interface,
        inputs=[camera_input, text_input],
        outputs=[image_output, text_output]
    )

# Launch the app
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://c1b8420af6ca1f7a3a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
